In [5]:
import os
import os

# Change to the correct directory where the EvoMan-Generalist folder is located
os.chdir('/Users/christophlaute/Evolutionary/Group-Assignment-Evolutionary-Computing/EvoMan-Generalist')

# Verify the current working directory
print("Current working directory:", os.getcwd())

print("Current working directory:", os.getcwd())

import numpy as np
import random
from evoman.environment import Environment
from demo_controller import player_controller
import os


Current working directory: /Users/christophlaute/Evolutionary/Group-Assignment-Evolutionary-Computing/EvoMan-Generalist
Current working directory: /Users/christophlaute/Evolutionary/Group-Assignment-Evolutionary-Computing/EvoMan-Generalist
pygame 2.6.0 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:

n_hidden_neurons=10
#n_weights = (env.get_num_sensors() + 1) * n_hidden_neurons + (n_hidden_neurons + 1) * 5
n_weights=265
experiment_name = 'generalist1'
if not os.path.exists(experiment_name):
    os.makedirs(experiment_name)

In [7]:
def initialize_population(population_size, n_vars):
    population=[]
    individual_mutation_rates=[]
    for i in range(population_size):
        population.append(np.random.uniform(-1, 1, n_vars)) 
    return np.array(population)

pop=initialize_population(3,265) # for debugging

def simulation(ind,enemies=[2,4,6]): # just for debugging purpose set enemies as 2, 4, 6
    ''' simulates the environment and evaluates the fitness score against group of enemies
    returns fitness_score: list of fitness scores against each enemy and also average fitness too
    basically fitness_score[0]= fitness score upon enemies[0] and so on 
    and fitness_score[4] = mean(fitness_score) - standard deviation(fitness_score)'''

    individual_scores=[] 
    for i in range(len(enemies)):
      env = Environment(experiment_name=experiment_name,
                  enemies=[enemies[i]],
                  playermode="ai",
                  player_controller=player_controller(n_hidden_neurons),
                  enemymode="static",
                  level=2,
                  speed="fastest",
                  visuals=False)
      f,p,e,t = env.play(pcont=ind)
      individual_scores.append(f)
    
    avg_fitness=env.cons_multi(np.array(individual_scores))
    individual_scores.append(avg_fitness)

    return individual_scores


def evaluation(population, enemies:list):
   '''calls the simulation function for each individual in population 
      returns np array of fitness scores for whole population'''
   
   fitness_scores=[]
   for ind in population:
      individual_scores = simulation(ind,enemies)
      fitness_scores.append(individual_scores)

   return np.array(fitness_scores)
    
evaluation(pop,[2,4,6]) # debugging




      


MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.


array([[-5.79605775, -4.99043259, -5.17614973, -5.66532946],
       [-5.95842469, -5.96100534, -5.50938834, -6.02189477],
       [-5.79605775, -5.42495002, -4.6443909 , -5.76843497]])

In [26]:
def one_dimensional_hc(fitness_values,reference_point):
    """calculates the hypervolume in the case that only one dimenstion is left"""
    sort_fit_val= fitness_values[fitness_values.argsort()]
    hyper_volume=0
    hyper_volume = np.sum(reference_point - sort_fit_val)
    return hyper_volume

def multi_dimensional_hc(fitness_values,reference_point):
    """recursively applies the hypervolume and computes the total hypervolume"""
    sort_fit_val= fitness_values[fitness_values[:,0].argsort()]
    hypervolume=0
    #computes hypervolume by adding contributions and multiplying by subvolumes
    for index,individual in enumerate(sort_fit_val[:,0]):
        cont = reference_point-individual
        fitness_copy = np.copy(sort_fit_val)
        new_fitness = np.delete(fitness_copy,0,axis = 1)
        sub_hyp = hypervolume_calculation(new_fitness[index:],reference_point)
        hypervolume += cont*sub_hyp
    return hypervolume

def hypervolume_calculation(fitness_values,reference_point):
    """applies the wfg algorithm to compute the hypervolume"""
    #sort individuals by fitness for first objective
    hypervolume=0
    #if only one dimension left
    if fitness_values.shape[1]==1:
        hypervolume=one_dimensional_hc(fitness_values,reference_point)
        return hypervolume

    #calculate contrubution to hypervolume for each individual
    hypervolume=multi_dimensional_hc(fitness_values,reference_point)
    return hypervolume


pop=initialize_population(3,265) # for debugging

fitness_values=evaluation(pop,[2,4,6])

hv=hypervolume_calculation(fitness_values,-5)
print(hv)



        



MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.

MESSAGE: Pygame initialized for simulation.
56056.50541734781
